In [1]:
import requests
from lxml import etree
import re
import json
import csv
import urllib3
from bs4 import BeautifulSoup
import time
import pandas as pd
import os
import shutil
import zipfile
urllib3.disable_warnings()

In [2]:
def create_csv(file_name):
    with open(file_name+".csv", "w", encoding="utf-8-sig", newline="") as f:
        csv_f = csv.writer(f)
        csv_f.writerow(columns)

def save_data(data, file_name):
    with open(file_name+".csv", "a", encoding="utf-8-sig", newline="") as f:
        csv_f = csv.writer(f)
        csv_f.writerow([data.get(key) for key in columns])

def extract_data(resp, file_name, skip_first=False):
    soup = BeautifulSoup(resp.text, "lxml")
    
    #------------------------------
    base = base = soup.find_all("div", class_="inner")[1:-1]
    #------------------------------
    
    for n, each in enumerate(base):
        extract_detail(each, file_name)
    
    #------------------------------
    #print(re.findall(r"\d+\.?\d*", soup.find("span", class_="page current").text)[0])
    #------------------------------
    
    #------------------------------
    pages = max_pages
    #------------------------------
    
    return pages

def extract_all_data(ori_url, sup):
    resp = requests.get(ori_url, headers=headers, verify=False)
    if resp.ok:
        total_pages = extract_data(resp, sup, skip_first=True)
    else:
        print('''The connection to the website failed, please check the url or anti-crawler.''')
    
    #------------------------------
    for p in range(2, int(total_pages)+1):
        time.sleep(3)

        #------------------------------
        url = ori_url+"?page={}".format(p)
        #------------------------------

        resp = requests.get(url, headers=headers, verify=False)
        if resp.ok:
            try:
                extract_data(resp, sup)
            except:
                break
    #------------------------------
            
def extract_detail(each, file_name=False):
    
    #------------------------------
    brand = "lakepajamas"
    product = each.find("span", class_="title uppercase futura").text.strip("\n")
    prices = "$"+re.findall(r"\d+\.?\d*", each.find("span", class_="price floatright").text)[0]
    ori_price = now_price = prices
    img_link = "https:"+each.find("img").get("src")
    product_link = "https://lakepajamas.com" + each.find("a").get("href")
    #------------------------------
    
    if file_name:
        save_data(dict(zip(columns, [brand, product, ori_price, now_price, img_link, product_link])), file_name)   

def download_img(img_url, file_name):
    resq = requests.get(img_url)
    with open(file_name, "wb") as f:
        f.write(resq.content)
        
def get_img(ori_file, sup):
    df = pd.read_csv(ori_file,encoding="utf-8")
    os.mkdir(sup+"_images")
    for n, url in df["image url"].iteritems():
        download_img(url,"./{}/".format(sup+"_images")+url.split("/")[-1].split("?")[0])
        if n%5==0: time.sleep(1)
            
def compress_file(zipfilename, dirname):
    if os.path.isfile(dirname):
        with zipfile.ZipFile(zipfilename, 'w') as z:
            z.write(dirname)
    else:
        with zipfile.ZipFile(zipfilename, 'w') as z:
            for root, dirs, files in os.walk(dirname):
                for single_file in files:
                    if single_file != zipfilename:
                        filepath = os.path.join(root, single_file)
                        z.write(filepath)

def mkdir(base_url):
    tar_path = base_url.split(".com")[0].strip("https://")
    if "www" in tar_path:
        tar_path = tar_path.split(".")[-1]
    isExists=os.path.exists(tar_path)
    if not isExists:
        os.mkdir(tar_path)
        print("parent folder created")
    return os.chdir(tar_path)

def folder_exc(folder_name):
    compress_file(folder_name+".zip", folder_name)
    shutil.rmtree(folder_name)
    return os.chdir(os.path.pardir)

def main(base_url, sup, maxi_pages):
    mkdir(base_url)
    url = base_url.format(sup)
    global columns, headers, max_pages
    max_pages = maxi_pages
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleW'
                      'ebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36',
        'accept-encoding': 'gzip, deflate, br',
    }
    columns = ["brand","product","original price","current price","image url","product url"]
    create_csv(sup)
    extract_all_data(url, sup)
    get_img(sup+".csv", sup)
    folder_exc(sup+"_images")

In [4]:
if __name__ == "__main__":
    for i in ["men"]:
        main("https://lakepajamas.com/collections/{}", i, 1)

parent folder created


----------------------------the following is for purpose of testing-------------------------

In [4]:
headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleW'
                      'ebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36',
        'accept-encoding': 'gzip, deflate, br',
    }

In [5]:
resp = requests.get("https://lakepajamas.com/collections/sleep", headers=headers, verify=False)

In [6]:
resp.ok

True

In [7]:
soup = BeautifulSoup(resp.text, "lxml")

In [6]:
re.findall(r"\d+\.?\d*", soup.find("span", class_="page current").text)[0]

'1'

In [8]:
base = soup.find_all("div", class_="inner")

In [14]:
[1,2,3][1:2]

[2]

In [12]:
each = base[90]

IndexError: list index out of range

In [10]:
each

<div class="inner centertext">
<div class="table">
<div class="cell">
<div class="title">Let's be friends.</div>
<div class="text">
                Sign up for our newsletter and receive 10%  <br/>
                off your next order. You'll be the first to<br/>
                know about new collections and special offers.
              </div>
<div class="form clearfix">
<!--<form action="//lakeweekendwear.us9.list-manage.com/subscribe/post?u=ac1214824a787d2c222326e7d&amp;id=6b6bebdac5" method="post" id="mc-embedded-subscribe-form" name="mc-embedded-subscribe-form" class="validate" target="_blank" novalidate>
                    <input type="text" name="EMAIL" class="futura floatleft" value="Your email address" onblur="if(this.value == '') { this.value='Your email address'}" onfocus="if (this.value == 'Your email address') {this.value=''}" />
                    <div style="position: absolute; left: -5000px;"><input type="text" name="b_ac1214824a787d2c222326e7d_6b6bebdac5" tabindex="-

In [24]:
#brand = each.find("span", class_="faux-link").text
#brand

'Long Pyjama Set'

In [19]:
each.find("span", class_="title uppercase futura").text.strip("\n")

'Pima Shorts Set in Hydrangea'

In [20]:
"$"+re.findall(r"\d+\.?\d*", each.find("span", class_="price floatright").text)[0]

'$84'

In [22]:
"https://lakepajamas.com" + each.find("a").get("href")

'https://lakepajamas.com/collections/sleep/products/hydrangea-shorts-set'

In [28]:
"https:"+each.find("img").get("src")

'https://cdn.shopify.com/s/files/1/0505/6125/products/shorts_hydrangea.jpg?v=1590771055'

In [ ]:
brand = "lakepajamas"
product = each.find("span", class_="title uppercase futura").text.strip("\n")
prices = "$"+re.findall(r"\d+\.?\d*", each.find("span", class_="price floatright").text)[0]
ori_price = now_price = prices
img_link = "https:"+each.find("img").get("src")
product_link = "https://lakepajamas.com" + each.find("a").get("href")

In [27]:
img_soup = BeautifulSoup(requests.get(product_link, "lxml")

In [32]:
img_soup.find("a", class_="slick-image").find("img").get("src")

'https://www.princessetamtam.com/dw/image/v2/ABBK_PRD/on/demandware.static/-/Sites-ptt-master/default/dw0727d5b2/MDODO293_1950_PF1.jpg?sw=700&sh=865&sm=cut'

In [27]:
n = 0
for folder in [i for i in os.listdir() if os.path.isdir(i)]:
    for f in os.listdir(folder):
        if "csv" in f:
            n += pd.read_csv(folder+"/"+f).shape[0]
print("Total number:", n)

Total number: 2003
